In [15]:
import os
import numpy as np
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from matplotlib.image import imread
from tensorflow.keras.metrics import AUC as auc

<h3>Setup</h3>

<h3>PreProcessing</h3>

In [9]:
from core import preprocessing

In [10]:
data_dir = preprocessing.root_data_dir
test_path = preprocessing.test_path
train_path = preprocessing.train_path
val_path = preprocessing.validation_path

image_shape = (300, 300, 3)

In [11]:
from tensorflow.keras.preprocessing.image import ImageDataGenerator

In [12]:
image_gen = ImageDataGenerator(fill_mode='nearest')
image_gen.flow_from_directory(train_path)
image_gen.flow_from_directory(test_path)
image_gen.flow_from_directory(val_path)

Found 15504 images belonging to 2 classes.
Found 6241 images belonging to 2 classes.
Found 16 images belonging to 2 classes.


<h2>Building a Model</h2>

In [13]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Conv2D, MaxPool2D, Dropout, Flatten, Layer, concatenate, GlobalAveragePooling2D,Activation, Softmax
from tensorflow.keras.metrics import Accuracy, AUC, Recall, Precision

In [14]:
class fire_module(Layer):
    
    def __init__(self, squeeze_size=16, expand_size=64):
        super(fire_module, self).__init__()
        self.squeeze = Conv2D(filters=squeeze_size, kernel_size=(1,1), padding='valid', \
                              activation='relu', name="sq1x1")
        self.exp1_1 = Conv2D(filters=expand_size, kernel_size=(1,1), padding='valid', \
                             activation='relu', name="exp1x1")
        self.exp3_3 = Conv2D(filters=expand_size, kernel_size=(3,3), padding='same', \
                             activation='relu', name="exp3x3")
    
    def call(self, input):
        squeezed_value = self.squeeze(input)
        exp1_1_value = self.exp1_1(squeezed_value)
        exp3_3_value = self.exp3_3(squeezed_value)
        return concatenate([exp1_1_value, exp3_3_value], axis=-1, name='concat')
    
    def get_config(self):
        config = super().get_config().copy()
        config.update({
            'squeeze': self.squeeze,
            'exp1_1': self.exp1_1,
            'exp3_3': self.exp3_3
        })
        return config

In [15]:
def build_model(expand_values, pooling_kernal):
    model = Sequential()
    model.add(Conv2D(filters=8, kernel_size=(3,3), input_shape=image_shape, activation='relu'))
    
    for expand_value in expand_values:
        model.add(fire_module(int((3*expand_value)/4),expand_value))
        model.add(MaxPool2D(pool_size=pooling_kernal))
    
    model.add(Conv2D(filters=1, kernel_size=(1,1), activation='sigmoid'))
    model.add(GlobalAveragePooling2D())

    model.compile(loss=loss_param, optimizer=optimizer_param,
                metrics=[AUC()])
    return model

In [16]:
loss_param = 'binary_crossentropy'
optimizer_param = 'adam'
stop_monitor = 'val_auc'
stop_mode = 'max'
stop_patience = 2

<h2>Regularization</h2>

In [17]:
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.utils import normalize

In [18]:
def regularize(batch_size):
    data = {}
    data['early_stop'] = EarlyStopping(monitor=stop_monitor, mode=stop_mode, patience=stop_patience)
    data['train_image_gen'] = image_gen.flow_from_directory(train_path,
                                               target_size = image_shape[:2],
                                               batch_size = batch_size,
                                               class_mode = 'binary')
    data['test_image_gen'] = image_gen.flow_from_directory(test_path,
                                               target_size=image_shape[:2],
                                               batch_size = batch_size,
                                               class_mode='binary',
                                               shuffle=False)# Don't want to shuffle test data and lose labels
    return data

<h2>Recording</h2>

In [19]:
def create_record(model, batch_size):
    losses = model.history.history
    losses['loss'] = np.asarray(losses['loss'])
    losses['val_loss'] = np.asarray(losses['val_loss'])
    final_number_of_epochs = len(losses['loss'])
    min_loss = losses['loss'].min()
    mean_loss = losses['loss'].mean()
    final_loss = losses['loss'][-1]
    min_val_loss = losses['val_loss'].min()
    mean_val_loss = losses['val_loss'].mean()
    final_val_loss = losses['val_loss'][-1]

    output = []
    model.summary(print_fn=lambda line: output.append(line))
    summary = str(output).strip('[]')

    record = {
        'Epochs': final_number_of_epochs,
        'Batch_Size': batch_size,
        'Loss_Func': loss_param,
        'Optimizer': optimizer_param,
        'Early_Stop_Monitor': stop_monitor,
        'Early_Stop_Patience': stop_patience,
        'Min_Loss': min_loss,
        'Mean_Loss': mean_loss,
        'Final_Loss': final_loss,
        'Min_Val_Loss': min_val_loss,
        'Mean_Val_Loss': mean_val_loss,
        'Final_Val_Loss': final_val_loss,
        'Model': summary
    }

    new_data = pd.DataFrame(record, index=[0])

    if os.path.exists('results.csv'):
        df_records = pd.read_csv('results.csv')
        df_records = df_records.append(new_data)
    else:
        df_records = pd.DataFrame(new_data)

    df_records.to_csv('results.csv', float_format='%g')

<h2>Running the Model</h2>

In [20]:
def run_test(expand_values, pooling_kernel, batch_size):
    model = build_model(expand_values, pooling_kernel)
    data = regularize(batch_size)
    return model, model.fit(data['train_image_gen'], epochs=1,
                             validation_data=data['test_image_gen'],
                    callbacks=[data['early_stop']]
                   )

In [ ]:
#Kelsey
'''
Dont run yet 
TODO: make run_test take in data to work on so this test can work with un processed data
'''
batch_size = 64
model, results = run_test([8],(4,4),batch_size)
print(model.summary)
create_record(model,batch_size)

metrics = pd.DataFrame(model.history.history)
metrics.plot()
metrics[['auc', 'val_auc']].plot()

model.save('model1.h5')#change this for every test

Found 15504 images belonging to 2 classes.
Found 624 images belonging to 2 classes.


In [ ]:
#Kelsey
batch_size = 64
model, results = run_test([8,16],(3,3),batch_size)
print(model.summary)
create_record(model,batch_size)

metrics = pd.DataFrame(model.history.history)
metrics.plot()
metrics[['auc', 'val_auc']].plot()

model.save('model2.h5')#change this for every test

In [ ]:
#George
batch_size = 64
model, results = run_test([8,16,24],(3,3),batch_size)
print(model.summary)
create_record(model,batch_size)

metrics = pd.DataFrame(model.history.history)
metrics.plot()
metrics[['auc', 'val_auc']].plot()

model.save('model3.h5')#change this for every test

In [ ]:
#Peter
batch_size = 64
model, results = run_test([8,16,24,32],(3,3),batch_size)
print(model.summary)
create_record(model,batch_size)

metrics = pd.DataFrame(model.history.history)
metrics.plot()
metrics[['auc', 'val_auc']].plot()

model.save('model4.h5')#change this for every test

In [ ]:
#George
batch_size = 32
model = run_test([8,16,24],(3,3),batch_size)
print(model.summary)
create_record(model,batch_size)

metrics = pd.DataFrame(model.history.history)
metrics.plot()
metrics[['auc', 'val_auc']].plot()

model.save('model5.h5')#change this for every test

In [ ]:
#Peter
batch_size = 32
model = run_test([8,16,24,32],(3,3),batch_size)
print(model.summary)
create_record(model,batch_size)

metrics = pd.DataFrame(model.history.history)
metrics.plot()
metrics[['auc', 'val_auc']].plot()

model.save('model6.h5')#change this for every test

In [ ]:
#after first ones
#George
batch_size = 64
model = run_test([8,16,24],(2,2),batch_size)
print(model.summary)
create_record(model,batch_size)

metrics = pd.DataFrame(model.history.history)
metrics.plot()
metrics[['auc', 'val_auc']].plot()

model.save('model7.h5')#change this for every test

In [ ]:
#Kelsey
batch_size = 64
model = run_test([8,16,24,32],(2,2),batch_size)
print(model.summary)
create_record(model,batch_size)

metrics = pd.DataFrame(model.history.history)
metrics.plot()
metrics[['auc', 'val_auc']].plot()

model.save('model8.h5')#change this for every test

In [ ]:
#Peter
batch_size = 64
model = run_test([8,16,24,32,40],(2,2),batch_size)
print(model.summary)
create_record(model,batch_size)

metrics = pd.DataFrame(model.history.history)
metrics.plot()
metrics[['auc', 'val_auc']].plot()

model.save('model9.h5')#change this for every test